<a href="https://colab.research.google.com/github/xuwenwu/molssi/blob/main/tutorials/en/1_2_atomistic_simulation_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notion

This notebook is the modified notebook of [AtomisticSimulationTutorial](https://docs.matlantis.com/atomistic-simulation-tutorial/en/) to run on Google Colab. We only provide executable parts with general library.

# Atomistic simulation introduction

What is "atomistic simulation", and what can be done with it?
This section provides an overview.

Various properties of materials can be explained at the atomic level.
For example, mechanical properties (elastic constants, Young's modulus, etc.), thermophysical properties (specific heat, etc.), viscosity, chemical reactions, etc.
Atomistic simulations can be used to reproduce how atoms move or to analyze how atoms are arranged in nature.

We will explain the detail gradually, but for now, let's start with the following code.

## Initial setup

In [ ]:
!pip install ase==3.23.0 asap3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.8 MB/s eta 0:00:00


In [ ]:
import os

from IPython.display import HTML, Image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation

from ase import Atoms
from ase.build import make_supercell
from ase.io import write
from ase.visualize import view

os.makedirs("output", exist_ok=True)

def view_x3d(atoms, idx=0):
    if isinstance(atoms[0], Atoms):
        # Assume this is a trajectory or struct list
        if (len(atoms) <= idx):
                print(f"The specified index exceeds the length of the trajectory. The length of the trajectory is {len(atoms)}.")
        return view(atoms[idx], viewer="x3d")
    else:
        return view(atoms, viewer="x3d")


def view_ase_atoms(atoms, rotation="0x,0y,0z", figsize=(4, 4), title="", scale=100):
    fig, ax = plt.subplots(figsize=figsize)
    write("output/tmp.png", atoms, rotation=rotation, scale=scale)
    img = mpimg.imread('output/tmp.png')
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
    plt.show()
    os.remove('output/tmp.png')
    return

def traj_to_apng(traj, rotation='30x,30y,30z'):
    imgs = []
    for atom in traj:
        # supercell = make_supercell(atom, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        # write('output/tmp.png', supercell, rotation=rotation, show_unit_cell=2)
        write("output/tmp.png", atom)
        img = mpimg.imread('output/tmp.png')
        imgs.append(img)
    os.remove('output/tmp.png')

    fig, ax = plt.subplots()

    def update(frame):
        img = imgs[frame]
        ax.clear()
        ax.imshow(img)
        return []

    ani = FuncAnimation(fig, update, frames=len(imgs), blit=True)
    plt.close()
    return HTML(ani.to_jshtml())

## Executing simple MD simulation example

This is a sample MD (Molecular Dynamics) simulation for Copper.
At this point, you do not need to understand the code at all. <br/>
Seeing is believing, so let's run it and visualize it first. <br/>
(By the end of this tutorial, you will be able to easily understand what this code is doing.)


In [ ]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.optimize import BFGS
from ase.io import Trajectory, read
from ase import units
from ase.build import bulk

from asap3 import EMT

calculator = EMT()

#atoms = read("../input/SiO2_mp-6930_conventional_standard.cif")
atoms = bulk("Cu", "fcc", a=3.64)
atoms.calc = calculator

opt = BFGS(atoms)
opt.run()

atoms = atoms * (6, 6, 6)
atoms.calc = calculator
# Set the momenta corresponding to T=1200K.
MaxwellBoltzmannDistribution(atoms, temperature_K=1200.0)
# Sets the center-of-mass momentum to zero.
Stationary(atoms)
# Run MD using the VelocityVerlet algorithm
dyn = VelocityVerlet(atoms, 1.0 * units.fs, trajectory="output/dyn.traj")

def print_dyn():
    print(f"Dyn  step: {dyn.get_number_of_steps(): >3}, energy: {atoms.get_total_energy():.3f}")

dyn.attach(print_dyn, interval=10)
dyn.run(100)

      Step     Time          Energy          fmax
BFGS:    0 23:58:19        0.005669        0.000000
Dyn  step:   0, energy: 37.020
Dyn  step:  10, energy: 37.021
Dyn  step:  20, energy: 37.023
Dyn  step:  30, energy: 37.027
Dyn  step:  40, energy: 37.028
Dyn  step:  50, energy: 37.025
Dyn  step:  60, energy: 37.023
Dyn  step:  70, energy: 37.023
Dyn  step:  80, energy: 37.024
Dyn  step:  90, energy: 37.024
Dyn  step: 100, energy: 37.024


True

In [ ]:
traj = Trajectory("output/dyn.traj")[::5]
view_x3d(traj)

In [ ]:
len(atoms)

216

In [ ]:
traj_to_apng(traj)

In [ ]:
atoms.get_cell_lengths_and_angles(), atoms.get_volume()

/tmp/ipykernel_63734/3485283998.py:1: DeprecationWarning: Please use atoms.cell.cellpar() instead
  atoms.get_cell_lengths_and_angles(), atoms.get_volume()


(array([15.4432121, 15.4432121, 15.4432121, 60.       , 60.       ,
        60.       ]),
 2604.3413760000008)

A technique called molecular dynamics is used to study the dynamics of atoms in Cu bulk at 1200K.

In the above example, we simulated 216 atoms.
The volume is about $2.6 \times 10^{-27}$m, and the length scale is about 11Å = ($15 \times 10^{-10}$m) for each axis, which is many orders of magnitude smaller than the 1m scale that we deal with in our daily lives.

One thing to consider here is that we cannot simulate on a computer the entire material we are dealing with on the same scale as it is.
Even materials on the order of grams have [Avogadro’s number](https://en.wikipedia.org/wiki/Avogadro_constant), i.e., atoms on the order of $10^{23}$, and the computer cannot handle such a large number of atoms.

Therefore, in atomistic simulations, it is necessary to perform appropriate **modeling** according to the phenomenon to create a simplified system that can be analyzed on a computer by extracting only the necessary elements to reproduce the desired phenomenon, rather than creating something exactly the same as the natural world.<br/>
There are various methods for modeling, and you will be able to do them by learning through this tutorial.

---

[Column] Modeling

For example, if you want to simulate the Earth, if you are interested in weather, you would need to focus your modeling on the Earth's surface atmosphere.
On the other hand, if you are interested in earthquakes, you would need to concentrate your modeling on the Earth's internal structure rather than the atmosphere.
If you want to simulate the environment on the surface of the earth, you might consider cutting out only a portion of a continent rather than the entire planet.

---

Again, let's look at the simulation results. The ones that appear here include the following.

- Atom
   - Each atom is specified by **element number**, has an **xyz coordinate value**, and has a **velocity**.
- **Cell**.
   - The box is represented by the cube in the above figure. It allows us to deal with systems in which this cell follows the **periodic boundary condition** indefinitely.

When dealing with a molecule floating in a vacuum,
the cell periodic boundary condition is not necessary.
For systems such as solids, which have a regular structure, the **periodic boundary condition** allows us to treat structures that extend infinitely in each of the x, y, and z axes (strictly speaking, the a, b, and c axes, which are the crystal axes).

The cell & periodic boundary condition are artificial concepts for the convenience of computation and modeling.
In reality, a crystal structure is considered to have a regular structure in the interior followed by another structure on the surface.
However the surface is in a very different state compared to interior and has special characteristics, making computation difficult in some cases.
Therefore, by artificially limiting our simulation to the world of cells and imposing the periodic boundary condition that the right end and the left end are connected, we can create a world that has no surface and continues repeatedly, so that we can deal with problems.

In the next section, we will learn how to handle these structures with Python programs and perform atomistic simulations.